In [ ]:
import numpy as np
import scipy.io.wavfile as wavfile
import matplotlib.pyplot as plt
import os

OUTPUT_DIR = "output"
IMAGES_DIR = os.path.join(OUTPUT_DIR, "images")
AUDIO_FILE = os.path.join(OUTPUT_DIR, "denoised_audio.wav")

os.makedirs(IMAGES_DIR, exist_ok=True)

def fourier_transform(signal, frequencies, sampled_times):
    num_freqs = len(frequencies)
    ft_result_real = np.zeros(num_freqs)
    ft_result_imag = np.zeros(num_freqs)
    
    for i, freq in enumerate(frequencies):
        cosine_term = signal * np.cos(2 * np.pi * freq * sampled_times)
        sine_term = signal * np.sin(2 * np.pi * freq * sampled_times)
        ft_result_real[i] = np.trapz(cosine_term, sampled_times)
        ft_result_imag[i] = -np.trapz(sine_term, sampled_times)
    
    return ft_result_real, ft_result_imag

def inverse_fourier_transform(ft_signal, frequencies, sampled_times):
    n = len(sampled_times)
    reconstructed_signal = np.zeros(n)
    
    for i, t in enumerate(sampled_times):
        real_part = np.sum(ft_signal[0] * np.cos(2 * np.pi * frequencies * t))
        imag_part = np.sum(ft_signal[1] * np.sin(2 * np.pi * frequencies * t))
        reconstructed_signal[i] = (real_part - imag_part) * (frequencies[1] - frequencies[0])
    
    return reconstructed_signal

def save_plot(x, y, title, xlabel, ylabel, filename):
    plt.figure(figsize=(12, 6))
    plt.plot(x, y)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.savefig(os.path.join(IMAGES_DIR, filename))
    plt.show()
    plt.close()

def filter_frequencies(ft_data, frequencies, magnitude_spectrum, threshold, window_width):
    filtered_ft_data = np.zeros_like(ft_data)
    filtered_ft_data[0] = ft_data[0].copy()
    filtered_ft_data[1] = ft_data[1].copy()
    
    peak_mask = magnitude_spectrum > threshold
    for i in range(len(peak_mask)):
        if peak_mask[i]:
            start_idx = max(0, i - window_width)
            end_idx = min(len(peak_mask), i + window_width)
            filtered_ft_data[0][start_idx:end_idx] *= 0.01
            filtered_ft_data[1][start_idx:end_idx] *= 0.01
    return filtered_ft_data

sample_rate, data = wavfile.read('buzzjc.wav')
data = data / np.max(np.abs(data))  
if len(data.shape) > 1:
    data = data.mean(axis=1)

time = np.linspace(0, len(data) / sample_rate, num=len(data))
save_plot(time, data, "Original Audio Signal (Time Domain)", "Time (s)", "Amplitude", "original_signal.png")

interval_step = 1
data_sampled = data[::interval_step]
sampled_times = np.linspace(0, len(data_sampled) / sample_rate, num=len(data_sampled))
frequencies = np.linspace(0, sample_rate / (2 * interval_step), num=len(data_sampled))

ft_data = fourier_transform(data_sampled, frequencies, sampled_times)
magnitude_spectrum = np.sqrt(ft_data[0]**2 + ft_data[1]**2)
save_plot(frequencies, magnitude_spectrum, "Frequency Spectrum of the Audio Signal", "Frequency (Hz)", "Magnitude", "frequency_spectrum.png")

MAGNITUDE_THRESHOLD = np.max(magnitude_spectrum) * 0.1
WINDOW_WIDTH = 100
filtered_ft_data = filter_frequencies(ft_data, frequencies, magnitude_spectrum, MAGNITUDE_THRESHOLD, WINDOW_WIDTH)

filtered_magnitude = np.sqrt(filtered_ft_data[0]**2 + filtered_ft_data[1]**2)
save_plot(frequencies, filtered_magnitude, "Filtered Frequency Spectrum (Unwanted Frequencies Removed)", "Frequency (Hz)", "Magnitude", "filtered_spectrum.png")

filtered_data = inverse_fourier_transform(filtered_ft_data, frequencies, sampled_times)

save_plot(sampled_times, filtered_data, "Reconstructed (Denoised) Audio Signal (Time Domain)", "Time (s)", "Amplitude", "reconstructed_signal.png")

filtered_data = np.int16(filtered_data / np.max(np.abs(filtered_data)) * 32767)
wavfile.write(AUDIO_FILE, sample_rate // interval_step, filtered_data)
